In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [43]:
df = pd.read_csv("final_train_set_w_measure_visit_obs.csv")
print("Dataset Shape: {}".format(df.shape))
df.head()

c:\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (457,1537,1553,1657,1733,1773,1901,1921,1933,1937,1949,1953,1961,1969,1973,1977,1981) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Dataset Shape: (49891, 1985)


,person_id,year_of_birth,month_of_birth,day_of_birth,location_id,gender_source_value,status,History of alcohol use,Peripheral oxygen saturation,Tobacco user,...,Rhinovirus RNA [Presence] in Unspecified specimen by NAA with probe detection_high,Rhinovirus RNA [Presence] in Unspecified specimen by NAA with probe detection_low,Human metapneumovirus RNA [Presence] in Unspecified specimen by NAA with probe detection_date,Human metapneumovirus RNA [Presence] in Unspecified specimen by NAA with probe detection_value,Human metapneumovirus RNA [Presence] in Unspecified specimen by NAA with probe detection_high,Human metapneumovirus RNA [Presence] in Unspecified specimen by NAA with probe detection_low,Muscle specific receptor tyrosine kinase Ab [Moles/volume] in Serum_date,Muscle specific receptor tyrosine kinase Ab [Moles/volume] in Serum_value,Muscle specific receptor tyrosine kinase Ab [Moles/volume] in Serum_high,Muscle specific receptor tyrosine kinase Ab [Moles/volume] in Serum_low
0,0,1948,3,7,6977,M,0.0,Yes,96.0,Never,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1932,9,25,1605,M,0.0,Yes,99.0,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1963,6,2,2242,F,0.0,No,95.0,Never,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1989,1,5,97,F,0.0,Yes,98.0,Never,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1951,7,25,3915,M,0.0,Yes,98.0,Never,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# Remove columns with number of nan greater than the threshold
feature_threshold = int(df.shape[0] * 0.5)
df = df.dropna(axis='columns', thresh=feature_threshold)
df.shape

(49891, 561)

In [45]:
# Remove observation with number of nan greater than the threshold
obs_threshold = int(df.shape[1] * 0.5)
df = df.dropna(axis='index', thresh=obs_threshold)
df.shape

(47971, 561)

In [46]:
# Check for columns which have hetrogenous datatypes and remove them
non_uniform_columns = []
for column in df.columns:
    if df.dtypes[column] == 'object':
        curr_type = type(df[column].iloc[0])
        if curr_type != str:
            df[column] = df[column].fillna(0)
        else:
            df[column] = df[column].fillna("0")
        for i in range(df.shape[0]):
            if type(df[column].iloc[i]) != curr_type:
                non_uniform_columns.append(column)
                break
df = df.drop(columns=non_uniform_columns)
df.shape

(47971, 524)

In [47]:
# Fill all numerical missing values with 0
df = df.fillna(0)

In [48]:
# Drop columns with constant values
constant_val_cols = []
for column in df.columns:
    if len(df[column].unique()) == 1:
        constant_val_cols.append(column)
df = df.drop(columns=constant_val_cols)
df.shape

(47971, 502)

In [49]:
# Test result, store since it might be dropped in correlation
results = df['status']
person_id = df['person_id']
df = df.drop(columns=['status', 'person_id'])

In [50]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
scalers = {}
encoders = {}
for column in df.columns:
    # If the column type is object (non-numerical) encode
    if df.dtypes[column] == 'object':
        encoders[column] = LabelEncoder()
        df[column] = encoders[column].fit_transform(df[column])
    # If the column type is numerical normalize
    else:
        scalers[column] = MinMaxScaler()
        df[column] = scalers[column].fit_transform(np.array(df[column]).reshape(-1, 1))

In [51]:
# https://www.dezyre.com/recipes/drop-out-highly-correlated-features-in-python
# First we determine the absolute values of correlation since we want to filter out both positive and negative correlation
absolute_correlation = df.corr().abs()
# Form an upper triangular matrix, since the correlation matrix is identical across diagonal
absolute_correlation_upper_triangle = absolute_correlation.where(np.triu(np.ones(absolute_correlation.shape), k=1).astype(np.bool))
# All features with a correlation higher than 0.95 to any other feature in the upper triangular correlation matrix is filtered
# Again Upper Triangular is vital since the diagonal represents correlation to itself which is always 1.0
columns_to_drop = []
for column in absolute_correlation_upper_triangle.columns:
    if any(absolute_correlation_upper_triangle[column] > 0.95):
        columns_to_drop.append(column)
corelation_filtered_df = df.drop(columns=columns_to_drop)
corelation_filtered_df.shape

(47971, 413)

In [52]:
# ANOVA F-value score based feature selection for our outcome feature
from sklearn.feature_selection import SelectKBest, f_classif
# Create and fit selector
feature_selector = SelectKBest(f_classif, k=50)
feature_selector.fit(corelation_filtered_df, results)
# Get columns chosen
cols = feature_selector.get_support(indices=True)
feature_selected_df = corelation_filtered_df.iloc[:,cols]

In [53]:
feature_selected_df.shape

(47971, 50)

In [54]:
feature_selected_df.insert(0, 'person_id', person_id)
feature_selected_df.insert(1, 'status', results)
feature_selected_df.head()

,person_id,Creatine kinase [Enzymatic activity/volume] in Serum or Plasma_high,Kappa light chains/Lambda light chains [Mass Ratio] in Serum by Immunoassay_date,Systolic blood pressure_value,Systolic blood pressure_high,Calcidiol [Mass/volume] in Serum or Plasma_value,Erythrocyte sedimentation rate by Westergren method_value,Prostate specific Ag [Mass/volume] in Serum or Plasma_date,Testosterone Free [Mass/volume] in Serum or Plasma_value,CD3+CD4+ (T4 helper) cells [#/volume] in Blood_value,...,"Glomerular filtration rate/1.73 sq M.predicted among non-blacks [Volume Rate/Area] in Serum, Plasma or Blood by Creatinine-based formula (MDRD)_date","Glomerular filtration rate/1.73 sq M.predicted among non-blacks [Volume Rate/Area] in Serum, Plasma or Blood by Creatinine-based formula (MDRD)_value",Base deficit in Arterial blood_date,Creatinine [Mass/volume] in Blood_value,Phosphate [Mass/volume] in Serum or Plasma_value,Oxygen content in Arterial blood_value,Lactate [Moles/volume] in Venous blood_value,Lactate [Moles/volume] in Venous blood_high,Hematocrit [Volume Fraction] of Blood by Automated count_value,Hematocrit [Volume Fraction] of Blood by Automated count_high
0,0,0.843077,2424,0.835366,0.000000,0.6640,0.904762,2464,0.600000,0.669725,...,2312,1.000000,852,0.00000,0.491803,0.497561,0.000000,0.00,0.000000,0.0
1,1,1.000000,3290,0.847561,0.750000,0.6560,0.000000,2596,0.082353,0.036697,...,2153,1.000000,2052,0.00000,0.836066,0.000000,0.000000,0.00,0.541667,0.9
2,2,1.000000,3323,0.000000,0.000000,0.1536,0.035714,2461,0.611765,0.330275,...,2739,1.000000,0,0.59375,0.704918,0.902439,0.185185,1.00,0.000000,0.0
3,3,1.000000,3284,0.890244,0.985714,0.5584,0.273810,1804,0.000000,0.293578,...,0,0.000000,1664,0.28125,0.491803,0.678049,0.259259,0.76,0.000000,0.0
4,4,1.000000,3004,0.676829,0.807143,0.1248,0.000000,1853,0.176471,0.880734,...,2813,0.916667,0,0.21875,0.000000,0.000000,0.074074,1.00,0.000000,0.0


In [57]:
# Export dataset
feature_selected_df.to_csv("pre_processed_dataset.csv", index=False)